# Higher-Order Functions
## Iteration Example
举了 Fibonacci 数列的例子


In [2]:
def fib(n):
    pre, cur = 0, 1
    k = 1
    while k < n:
        pre, cur = cur, pre + cur
        k = k + 1
    return cur

fib(20)

6765